In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import sys
sys.path.append('../learnMSA')
import numpy as np
from learnMSA import msa_hmm
from matplotlib import pyplot as plt
from src import Util

## Align Refinement

learnMSA leaves insertions unaligned. This is a problem, if long, important motifs (additional domains) are underrepresented in the sequences. 

Solution: Once a pHMM is trained, introduce a simple descision rule whether insertions should be recursively aligned. For each position $i$ where at least $k$ insertions longer than $t$ residues exists, realign those insertions (i.e. a horizontal and vertical slice of the input sequences) recursively.

## Find long insertions and align them separately

In [10]:
def find_long_insertions_and_write_slice(fasta_file, lens, starts, name, t = 20, k=2, verbose=True):
        """
        Finds insertions that have at least length t. If there are at least k of these sequences, writes them to file.
        Args: 
            lens, starts: Arrays of length n where n is the number of sequences. Indicate how long insertions are and where they start respectively.
            name: Identifier for the location of the slice.
        """
        at_least_t = lens >= t
        lengths = lens[at_least_t]
        if lengths.size > k:
            if verbose:
                print(f"Long insertions found at {name}: {lengths.size}.")
            which = np.squeeze(np.argwhere(at_least_t))
            start = starts[at_least_t]
            filename = f"tmp/slice_{name}"
            to_delete = []
            with open(filename, "w") as slice_file:
                for j in range(lengths.size):
                    aa_seq = fasta_file.aminoacid_seq_str(which[j])
                    segment = aa_seq[start[j] : start[j] + lengths[j]]
                    only_non_standard_aa = True
                    for aa in msa_hmm.fasta.alphabet[:20]:
                        if aa in segment:
                            only_non_standard_aa = False
                            break
                    if only_non_standard_aa:
                        to_delete.append(j)
                    else:
                        slice_file.write(">"+fasta_file.seq_ids[which[j]]+"\n")
                        slice_file.write(segment+"\n")
            which = np.delete(which, to_delete)
            return (which, filename)

In [26]:
!source activate snakeMSA ; famsa -h

FAMSA (Fast and Accurate Multiple Sequence Alignment) 
  version 2.2.2- (2022-10-09)
  S. Deorowicz, A. Debudaj-Grabysz, A. Gudys

Usage:
  famsa [options] <input_file> [<input_file_2>] <output_file>

Positional parameters:
  input_file, input_file_2 - input files in FASTA format; action depends on the number of input files:
      * one input - multiple sequence alignment (input gaps, if present, are removed prior the alignment),
      * two inputs - profile-profile aligment (input gaps are preserved).
      First input can be replaced with STDIN string to read from standard input.
  output_file - output file (pass STDOUT when writing to standard output); available outputs:
      * alignment in FASTA format,
      * guide tree in Newick format (-gt_export option specified),
      * distance matrix in CSV format (-dist_export option specified),

Options:
  -help - print this message
  -t <value> - no. of threads, 0 means all available (default: 0)
  -v - verbose mode, show timing inform

In [27]:
def make_aligned_insertions(alignment_model):
    data = alignment_model.metadata[alignment_model.best_model]
    num_seq = data.left_flank_len.shape[0]

    insertions_long = []
    for r in range(data.num_repeats):
        insertions_long.append([])
        for i in range(data.insertion_lens.shape[2]):
            ins_long = find_long_insertions_and_write_slice(alignment_model.fasta_file, data.insertion_lens[r, :, i], data.insertion_start[r, :, i], f"ins_{r}_{i}")
            insertions_long[-1].append(ins_long)
    left_flank_long = find_long_insertions_and_write_slice(alignment_model.fasta_file, data.left_flank_len, data.left_flank_start, "left_flank")
    right_flank_long = find_long_insertions_and_write_slice(alignment_model.fasta_file, data.right_flank_len, data.right_flank_start, "right_flank")
    unannotated_long = []
    for r in range(data.num_repeats-1):
        unannotated_long.append(find_long_insertions_and_write_slice(alignment_model.fasta_file, data.unannotated_segments_len[r], data.unannotated_segments_start[r], f"unannotated_{r}"))
        
    slice_files = []
    if left_flank_long is not None:
        slice_files.append("tmp/slice_left_flank")
    if right_flank_long is not None:
        slice_files.append("tmp/slice_right_flank")
    for r in range(data.num_repeats):
        for i in range(data.insertion_lens.shape[2]):
            if insertions_long[r][i] is not None:
                slice_files.append(f"tmp/slice_ins_{r}_{i}")
    for r in range(data.num_repeats-1):
        if unannotated_long[r] is not None:
            slice_files.append(f"tmp/slice_unannotated_{r}")
            
    #align insertions
    for slice_file in slice_files:
        #with learnMSA
        #config = msa_hmm.config.make_default(alignment_model.num_models)
        #msa_hmm.align.run_learnMSA(slice_file, slice_file+".aln", config, verbose=False)
        #with t_coffee regressive
        #!export MAX_N_PID_4_TCOFFEE=10000000 ; ~/bin/t_coffee -reg -seq {slice_file} -nseq 100 -tree mbed -method mafftginsi_msa -outfile {slice_file}.aln -quiet
        #with clustalo
        #!source activate clustalo ; clustalo --threads=32 -i {slice_file} -o {slice_file}.aln --force
        #with famsa
        !source activate snakeMSA ; famsa {slice_file} {slice_file}.aln
        
    #merge msa
    insertions_long = [[(x[0], msa_hmm.fasta.Fasta(x[1]+".aln", aligned=True)) if x is not None else None for x in repeats] for repeats in insertions_long]
    left_flank_long = (left_flank_long[0],  msa_hmm.fasta.Fasta(left_flank_long[1]+".aln", aligned=True)) if left_flank_long is not None else None
    right_flank_long = (right_flank_long[0],  msa_hmm.fasta.Fasta(right_flank_long[1]+".aln", aligned=True)) if right_flank_long is not None else None
    unannotated_long = [(x[0], msa_hmm.fasta.Fasta(x[1]+".aln", aligned=True)) if x is not None else None for x in unannotated_long]
    
    aligned_insertions = msa_hmm.alignment_model.AlignedInsertions(insertions_long, left_flank_long, right_flank_long, unannotated_long)
    return aligned_insertions

In [28]:
def runner_func(train_file, ref_file, out_file, config):
    alignment_model = msa_hmm.align.run_learnMSA(train_file, out_file+".default", config, verbose=True)
    aligned_insertions = make_aligned_insertions(alignment_model)
    alignment_model.to_file(out_file, alignment_model.best_model, aligned_insertions = aligned_insertions)
    return alignment_model, 0.

In [29]:
runner_func("data/homfam/train/Ald_Xan_dh_2.fasta", "", "test.Ald_Xan_dh_2.fasta", msa_hmm.config.make_default(10))

Training of 10 models on file Ald_Xan_dh_2.fasta
Configuration: 
{
num_models : 10
transitioner : ProfileHMMTransitioner(
 transition_init=
    {
    begin_to_match : DefaultEntry() , match_to_end : DefaultExit() , 
    match_to_match : DefaultMatchTransition(1) , match_to_insert : DefaultMatchTransition(-1) , 
    insert_to_match : Norm(0, 0.1) , insert_to_insert : Norm(-0.5, 0.1) , 
    match_to_delete : DefaultMatchTransition(-1) , delete_to_match : Norm(0, 0.1) , 
    delete_to_delete : Norm(-0.5, 0.1) , left_flank_loop : Norm(0, 0.1) , 
    left_flank_exit : Norm(-1, 0.1) , right_flank_loop : Norm(0, 0.1) , 
    right_flank_exit : Norm(-1, 0.1) , unannotated_segment_loop : Norm(0, 0.1) , 
    unannotated_segment_exit : Norm(-1, 0.1) , end_to_unannotated_segment : Norm(-9, 0.1) , 
    end_to_right_flank : Norm(0, 0.1) , end_to_terminal : Norm(0, 0.1)
    },
 flank_init=Const(0.0),
 prior=ProfileHMMTransitionPrior(match_comp=1, insert_comp=1, delete_comp=1, alpha_flank=7000, alpha_s

(<learnMSA.msa_hmm.AlignmentModel.AlignmentModel at 0x7ff5171c5970>, 0.0)

In [30]:
!id_list=$(sed -n '/^>/p' "data/homfam/refs/Ald_Xan_dh_2.ref" | sed 's/^.//'); export MAX_N_PID_4_TCOFFEE=10000000 ; ~/bin/t_coffee -other_pg seq_reformat -in test.Ald_Xan_dh_2.fasta -action +extract_seq_list "${id_list[@]}" +rm_gap > test/data/projection

HERE: 1dgja
HERE: 1hlra
HERE: 1fo4a
HERE: 1jrob
HERE: 1ffvb
HERE: 1n62b


In [32]:
!export MAX_N_PID_4_TCOFFEE=10000000 ; ~/bin/t_coffee -other_pg aln_compare -al1 data/homfam/refs/Ald_Xan_dh_2.ref -al2 test/data/projection -compare_mode sp

*****************************************************
seq1       seq2          Sim   [ALL]           Tot  
Ald_Xan_dh_2  6          31.8    82.8 [100.0]   [21304]


In [ ]:
name = "align_insertions_famsa"
Util.run_tests(runner_func, name, datasets=["homfam"])

INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/cys/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/DMRL_synthase/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/Stap_Strp_toxin/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/ghf5/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/myb_DNA-binding/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/ghf10/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/il8/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/cytb/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/kringle/model/assets
INFO:tensorflow:Assets written to: results/align_insertions_famsa/models/LIM/model/assets
INFO:tensorflow:Assets written to: results/align_insertion

In [9]:
!export MAX_N_PID_4_TCOFFEE=10000000 ; ~/bin/t_coffee -reg -seq tmp/slice_ins_0_51 -nseq 100 -tree mbed -method mafftginsi_msa -outfile tmp/slice_ins_0_51.aln -quiet

In [3]:
Util.get_score("align_insertions_t_coffee", "homfam")

83.90425531914892

In [6]:
Util.get_score("align_insertions_famsa", "homfam")

83.90744680851066

In [7]:
Util.get_score("align_insertions_clustalo", "homfam")

83.7489361702128

In [9]:
Util.get_score("reg", "homfam")

83.60106382978722